In [20]:
import re
from nltk.corpus.reader.indian import IndianCorpusView
import numpy as np
dataset = IndianCorpusView('Official-Docs-Data.txt','utf-8', True, True,)
data=[]

with dataset as lines:
    for line in lines:
        data.append(line)


# data preprocessing

In [21]:

sentence=[]
tags=[]
temp_sentence=[]
temp_tag=[]
tag_finder="[A-Z]+" 
word_finder='[^A-Z]+'
for word in data:
  # print(word[0][0])
  if word==[]:#eliminating the empty list
    continue
  else:
    tag=re.findall(tag_finder,word[0][0])
    word=re.findall(word_finder,word[0][0])
    if tag[0]=="NUM": #condition for finding the end of the sentence
      if temp_sentence==['-'] or []:
        temp_sentence=[]
        temp_tag=[]
        continue
      sentence.append(temp_sentence)
      tags.append(temp_tag)
      temp_sentence=[]
      temp_tag=[]
    else:
      temp_sentence.append(word[0][0:len(word[0])-1])
      temp_tag.append(tag[0])
    # print(tag,word)

In [22]:
from nltk import flatten
temp_tags=flatten(tags)
unique_tags=[]
for tag in temp_tags:
    if tag in unique_tags:
        pass
    else:
        if tag != []:#eliminating the empty tags
            unique_tags.append(tag)
        else:
            pass

In [23]:
index=[]
for x in unique_tags:
    for y in unique_tags:
        index.append((x,y))

#### calculating transition probability

In [24]:

#find how likely a tag is followed by another
#denominator
from collections import defaultdict
tag_counter=defaultdict(int)
tag_counts={}
all_tags=flatten(tags)
for unique in unique_tags:
    for tag in all_tags:
        if unique==tag:
            tag_counter[unique]
            tag_counter[unique]+=1

In [25]:
# numerator
i=0
while i<len(temp_tags):
    # print(temp_tags[i])
    i+=1

In [26]:
numerator=defaultdict(int)
for i in index:
    # print(i[0],i[1])
    j=0
    while j<len(temp_tags):
        try:
            if temp_tags[j]==i[0] and temp_tags[j+1]==i[1]:
                # print(temp_tags[j],temp_tags[j+1])
                numerator[i]+=1
            j+=1
        except IndexError:
            j+=1
            pass

In [27]:
transition_table=defaultdict(float)
for cell in index:
    # print(cell)
    # print(numerator[cell])
    # print(tag_counter[cell[0]])
    transition_table[cell]=numerator[cell]/tag_counter[cell[0]] 

#### calculating emission probability

In [28]:

words=[]
temp_word=[]
tag_finder="[A-Z]+" 
word_finder='[^A-Z]+'
for word in data:
  # print(word[0][0])
  if word==[]:#eliminating the empty list
    continue
  else:
    tag=re.findall(tag_finder,word[0][0])
    word=re.findall(word_finder,word[0][0])
    if tag[0]=="NUM": #condition for finding the end of the sentence
      if temp_sentence==['-'] or []:
        temp_word=[]
        continue
      words.append(temp_word)
      temp_word=[]
    else:
      temp_word.append((word[0][0:len(word[0])-1],tag[0]))

    # print(tag,word)

In [29]:
comb=defaultdict(int)
for word in words:
    ele=0
    while ele<len(word):
        # print(word[ele])
        comb[word[ele]]+=1
        ele+=1

In [30]:
emission_prob=defaultdict(float)
for key,values in comb.items():
    emission_prob[key]=values/tag_counter[key[1]]

checking how many words has more than one tag in the dataset

In [31]:
unique_words=[]
for word in words:
    ele=0
    while ele<len(word):
        if word[ele][0] not in unique_words:
            unique_words.append(word[ele][0])
        ele+=1

In [32]:

word_tag=[]
temp_sentence=[]
tag_finder="[A-Z]+" 
word_finder='[^A-Z]+'
for word in data:
  # print(word[0][0])
  if word==[]:#eliminating the empty list
    continue
  else:
    tag=re.findall(tag_finder,word[0][0])
    word=re.findall(word_finder,word[0][0])
    if tag[0]=="NUM": #condition for finding the end of the sentence
      if temp_sentence==['-'] or []:
        continue
    else:
      word_tag.append((word[0][0:len(word[0])-1],tag[0]))
    # print(tag,word)

In [33]:
temp1=defaultdict(list)
for word in unique_words:
    # print(word)
    for comb in word_tag:
        if comb[0]==word:
            if comb[1] not in temp1[word]:
                temp1[word].append(comb[1])

# Tagging

In [34]:
#tokenize every word 
#find all possible tags for each word
#calculate transition and emission prob
#product(transistion and emision) and select its best

In [35]:
import copy
temp=defaultdict(list)
def seq_gen(tagged):
    only_tag=[]
    for key,values in tagged.items(): # makes only one run
        only_tag.append(values)
    cp_only_tag = copy.deepcopy(only_tag)
    #find lengthy tag
    length=0
    for i in only_tag:
        if len(i)>length:
            length=len(i)
    for i in only_tag:
        if len(i)<length:
            n=length-len(i)
            for j in range(n):
                i.append(0)
    initial_combo=[]
    for i in range(length):
        for j in only_tag:
            initial_combo.append(j[i])
    initial_combo=flatten(initial_combo)
    initial_combo=np.array(initial_combo).reshape(length,len(tagged))
    for combo in initial_combo:
        if '0' in combo:
            # print(combo)
            for tag in range(len(combo)):
                if combo[tag]=='0':
                    possible_tags=cp_only_tag[tag]
                    # print("missing tag position",tag,"possible tags are",cp_only_tag[tag])
                    if len(possible_tags)==1: #if the possible tag is only one 
                        combo[tag]=cp_only_tag[tag][0]
                    else:
                        pass
    # print(initial_combo)
    return initial_combo

def prob_calc(sequence,ip):
    # print(sequence,ip)
    transition=[]
    for i in sequence:
        trans=[]
        for j in range(len(i)):
            try:
                trans.append((i[j],i[j+1]))
            except IndexError:
                pass
        transition.append(trans)
    for i in range(len(transition)):
        for j in range(len(transition[i])):
            # print(i,j)
            # print(transition[i][j])
            # print(transition_table[transition[i][j]])
            transition[i][j]=transition_table[transition[i][j]]
    
    emis=[]
    for i in sequence:
        t=[]
        new=zip(ip,i)
        for j in new:
            # print(j)
            t.append(j)
        emis.append(t)
    for i in range(len(emis)):
        for j in range(len(emis[i])):
            # print(i,j)
            # print(emis[i][j])
            # print(emission_prob[emis[i][j]])
            emis[i][j]=emission_prob[emis[i][j]]
    return emis,transition

def best_finder(transition,emision):
    final=[]
    for i in range(len(transition)):
        # print(transition[i])
        transition[i]=np.prod(transition[i])
    for i in range(len(emision)):
        # print(emision[i])
        emision[i]=np.prod(emision[i])
    for i in range(len(transition)):
        final.append(transition[i]*emision[i])
    return final.index(max(final))    
def tagger(ip):
    ip=ip.split(" ")
    ###for finding different tags that a word can take
    for word in ip:
        # print(word)
        for i in words:
            ele=0
            while ele<len(i):
                # print(i[ele])
                if word==i[ele][0]: # finding word match
                    if i[ele][1] not in temp[word]:
                        temp[word].append(i[ele][1])
                ele+=1
    # print("\n The possible tags that can be taken by the words in the sentence :\n ",temp)
    sequence=seq_gen(temp)
    # print("\n The possible tags that the given sentence can have is \n",sequence)
    # print(type(sequence))
    # print(sequence.shape)
    # print(sequence,ip)
    # print(type(sequence),type(ip))
    emision,transition=prob_calc(sequence,ip)
    best=best_finder(transition,emision)
    # print("best",best)
    print(ip)
    print(sequence[best])
    
tagger("ஜனவரி டிசம்பர் மாதம் வரையில் பின்வரும் கூட்டங்கள் நடாத்தப்பட்டன")


['ஜனவரி', 'டிசம்பர்', 'மாதம்', 'வரையில்', 'பின்வரும்', 'கூட்டங்கள்', 'நடாத்தப்பட்டன']
['PROPN' 'PROPN' 'NOUN' 'ADP' 'ADJ' 'NOUN' 'VERB']
